In [2]:
import pandas as pd

## Import and Inspect the file

The objective of this proration script is to breakdown a each data point with a start date and end date spanning multiple months into multiple entries labelled by the corresponding months. We inspect the raw dataset to look for the required columns

In [3]:
df = pd.read_csv('dummy_data.csv')
df

,contract_id,contract_name,customer_id,start_date,end_date,amount
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250
2,C003,Contract Name,S001,24/9/2022,17/5/2023,22560
3,C004,Some Contract,S003,28/2/2023,14/3/2023,1450
4,C005,JKL Contract,S004,17/5/2023,31/5/2023,1337


Note that some of the start date and end dates span across months. We want to answer the question: How much of the contract amount is attributed to the months that the contract has been running for. To do this, we need to find out how many days the contract has been running for each month, and pro-rate it accordingly.
We create a function that generates the data based on the start and end dates.

In [20]:
def split_by_month(start_date, end_date):
    date_range = pd.date_range(start_date, end_date, freq='D')
    counts = pd.Series(date_range).dt.to_period('M').value_counts().sort_index().reset_index()
    month_year = counts.iloc[:,0].astype(str).to_list()
    days_in_month = counts.iloc[:,1].to_list()
    total_days = sum(days_in_month)
    ratio = [i/total_days for i in days_in_month]
    return [month_year, days_in_month, ratio]

In [21]:
# Test case for the function:
split_by_month('2022-06-12', '2023-01-05')

[['2022-06',
  '2022-07',
  '2022-08',
  '2022-09',
  '2022-10',
  '2022-11',
  '2022-12',
  '2023-01'],
 [19, 31, 31, 30, 31, 30, 31, 5],
 [0.09134615384615384,
  0.14903846153846154,
  0.14903846153846154,
  0.14423076923076922,
  0.14903846153846154,
  0.14423076923076922,
  0.14903846153846154,
  0.02403846153846154]]

The function takes in a start and end date and returns a nested list containing the months, days in each month, and the ratio - the number of days of that month out of the entire duration of start and end dates

## Applying the function to the DataFrame, Exploding the lists to get extra rows

In [22]:
df['split_data'] = df.apply(lambda x: split_by_month(x['start_date'], x['end_date']), axis=1)
df

,contract_id,contract_name,customer_id,start_date,end_date,amount,split_data
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,"[[2022-12, 2023-01, 2023-02, 2023-03, 2023-04,..."
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,"[[2022-06, 2022-07, 2022-08, 2022-09], [2, 31,..."
2,C003,Contract Name,S001,24/9/2022,17/5/2023,22560,"[[2022-09, 2022-10, 2022-11, 2022-12, 2023-01,..."
3,C004,Some Contract,S003,28/2/2023,14/3/2023,1450,"[[2023-02, 2023-03], [1, 14], [0.0666666666666..."
4,C005,JKL Contract,S004,17/5/2023,31/5/2023,1337,"[[2023-05], [15], [1.0]]"


In [29]:
df[['month_year', 'days_in_month', 'ratio']] = pd.DataFrame(df['split_data'].to_list(), index=df.index)
df.drop('split_data', axis=1, inplace=True)
df

,contract_id,contract_name,customer_id,start_date,end_date,amount,month_year,days_in_month,ratio
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,"[2022-12, 2023-01, 2023-02, 2023-03, 2023-04, ...","[26, 31, 28, 31, 30, 1]","[0.17687074829931973, 0.2108843537414966, 0.19..."
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,"[2022-06, 2022-07, 2022-08, 2022-09]","[2, 31, 31, 21]","[0.023529411764705882, 0.36470588235294116, 0...."
2,C003,Contract Name,S001,24/9/2022,17/5/2023,22560,"[2022-09, 2022-10, 2022-11, 2022-12, 2023-01, ...","[7, 31, 30, 31, 31, 28, 31, 30, 17]","[0.029661016949152543, 0.13135593220338984, 0...."
3,C004,Some Contract,S003,28/2/2023,14/3/2023,1450,"[2023-02, 2023-03]","[1, 14]","[0.06666666666666667, 0.9333333333333333]"
4,C005,JKL Contract,S004,17/5/2023,31/5/2023,1337,[2023-05],[15],[1.0]


In [30]:
df = df.explode(['month_year', 'days_in_month', 'ratio'])
df

,contract_id,contract_name,customer_id,start_date,end_date,amount,month_year,days_in_month,ratio
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2022-12,26,0.176871
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-01,31,0.210884
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-02,28,0.190476
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-03,31,0.210884
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-04,30,0.204082
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-05,1,0.006803
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-06,2,0.023529
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-07,31,0.364706
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-08,31,0.364706
1,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-09,21,0.247059


## Clean-up and further analysis

At this point, the objective has been completed and anything else is just a matter of cleaning up based on how you want to prepare the data for further analysis.
You could: Remove unnecessary columns, multiply the ratio by the contract amount to get the pro-rated amount, reset the index, aggregate the data for further analysis

In [35]:
df['prorated_amount'] = df['amount'] * df['ratio']
df.reset_index(inplace=True, drop=True)
df.drop(['days_in_month', 'ratio'], axis=1, inplace=True)
df

,contract_id,contract_name,customer_id,start_date,end_date,amount,month_year,prorated_amount
0,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2022-12,2087.07483
1,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-01,2488.435374
2,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-02,2247.619048
3,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-03,2488.435374
4,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-04,2408.163265
5,C001,ABC Contract,S001,12/6/2022,5/1/2023,11800,2023-05,80.272109
6,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-06,123.529412
7,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-07,1914.705882
8,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-08,1914.705882
9,C002,XYZ Contract,S002,29/6/2022,21/9/2022,5250,2022-09,1297.058824
